In [16]:
import pandas as pd

In [17]:
import spacy
import en_core_web_sm

In [18]:
df = pd.read_csv('india-news-headlines.csv')


In [19]:
df

,publish_date,headline_category,headline_text
0,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
1,20010102,unknown,Fissures in Hurriyat over Pak visit
2,20010102,unknown,America's unwanted heading for India?
3,20010102,unknown,For bigwigs; it is destination Goa
4,20010102,unknown,Extra buses to clear tourist traffic
...,...,...,...
3650965,20220331,city.srinagar,J&K sacks 2 cops; 3 other employees over terro...
3650966,20220331,entertainment.hindi.bollywood,Ranbir Kapoor says 'Rishi Kapoor enjoyed his a...
3650967,20220331,city.trichy,As Covid-19 cases drop to nil in southern dist...
3650968,20220331,city.erode,Tamil Nadu sees marginal rise of Covid cases w...


In [20]:
df_time_period = df[(df["publish_date"]>20130000) & (df["publish_date"]<20140516)]
df_time_period

,publish_date,headline_category,headline_text
1531086,20130101,life-style.food.recipes,Appetizer recipe: Corn & Basil Fingers
1531087,20130101,life-style.relationships.man-woman,Men taking to cooking like never before
1531088,20130101,life-style.health-fitness.health-news,Theatre can be therapeutic
1531089,20130101,unknown,Should you buy a resale flat?
1531090,20130101,life-style.relationships.man-woman,How to not get raped
...,...,...,...
1879111,20140515,city.bhopal,Inaugural of event to mark birth anniversary o...
1879112,20140515,entertainment.events.bhopal,Play Drishtidaan staged at the third day of Ra...
1879113,20140515,city.bhopal,Play Drishtidaan staged at the third day of Ra...
1879114,20140515,entertainment.events.bhopal,A city-based band; Bandishein performs at Rabi...


In [24]:
f = open('political_cats.txt', 'r')

textblob = f.read()
cats = textblob.split("\n")

In [25]:
relevant_df = df_time_period[ df_time_period['headline_category'].isin(cats)]

In [26]:
relevant_df

,publish_date,headline_category,headline_text
1531135,20130101,india,How can we trust this government; questions Ni...
1531146,20130101,india,Rapper's concert scrapped after online protest...
1531162,20130101,india,Govt prunes cash transfers to 20 districts fro...
1531173,20130101,india,Delhi gang-rape case: Grieving family says mar...
1531174,20130101,india,Delhi gang-rape case: 80 witnesses in chargesheet
...,...,...,...
1879009,20140515,india,Gadkari was never focus of our probe; I-T depa...
1879013,20140515,india,Narendra Modi chairs Gujarat cabinet meeting; ...
1879018,20140515,india,Average Indian male consumes 33 litres of alco...
1879024,20140515,india,Sulking Sushma flies to Bhopal says not angry


In [27]:
series = df.groupby("headline_category")["headline_category"].count()
# df.groupby('News Portal')['Positive Sentiment Score'].median()

series = series.sort_values()

print(series)
series.to_csv("Cat Counts.csv")

headline_category
young-turks                                   8
nepal-india-earthquake.opinion                8
sports.rio-2016-olympics.news.sailing         8
himachal-pradesh                              8
best-products.beauty.grooming                 8
                                          ...  
business.india-business                  121195
city.delhi                               137648
city.mumbai                              150451
unknown                                  209583
india                                    297491
Name: headline_category, Length: 1041, dtype: int64


In [28]:
cate = pd.read_csv("Cat Counts.csv")

cate[cate["headline_category.1"] == cate["headline_category.1"].max()]

,headline_category,headline_category.1
1040,india,297491


In [29]:
relevant_df

,publish_date,headline_category,headline_text
1531135,20130101,india,How can we trust this government; questions Ni...
1531146,20130101,india,Rapper's concert scrapped after online protest...
1531162,20130101,india,Govt prunes cash transfers to 20 districts fro...
1531173,20130101,india,Delhi gang-rape case: Grieving family says mar...
1531174,20130101,india,Delhi gang-rape case: 80 witnesses in chargesheet
...,...,...,...
1879009,20140515,india,Gadkari was never focus of our probe; I-T depa...
1879013,20140515,india,Narendra Modi chairs Gujarat cabinet meeting; ...
1879018,20140515,india,Average Indian male consumes 33 litres of alco...
1879024,20140515,india,Sulking Sushma flies to Bhopal says not angry


In [30]:

nlp = en_core_web_sm.load()

texts = ''.join(str(relevant_df.headline_text.tolist()))

nodes = []
entities = ["ORG", "PERSON", "NORP", "EVENT", "LAW"]



In [31]:
print(len(nodes))

0


In [13]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax

MODEL = f"cardiffnlp/xlm-twitter-politics-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)



In [32]:
relevant_df["Positive Sentiment Score"] = 0
relevant_df["Neutral Sentiment Score"] = 0
relevant_df["Negative Sentiment Score"] = 0

/var/folders/4p/rylg4l1j585g32zrfkvlzbgh0000gn/T/ipykernel_12510/3015169735.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_df["Positive Sentiment Score"] = 0
/var/folders/4p/rylg4l1j585g32zrfkvlzbgh0000gn/T/ipykernel_12510/3015169735.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_df["Neutral Sentiment Score"] = 0
/var/folders/4p/rylg4l1j585g32zrfkvlzbgh0000gn/T/ipykernel_12510/3015169735.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

In [33]:
for index, row in relevant_df.iterrows():
  text = row["headline_text"]
  encoded_input = tokenizer.encode(text, padding=True, truncation=True,max_length=50, add_special_tokens = True, return_tensors='pt')

  output = model(encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  
  ranking = np.argsort(scores)
  for i in range(scores.shape[0]):
    s = scores[ranking[i]]
    if i==0:
      relevant_df.at[index,"Positive Sentiment Score"] = s
    if i==1:
      relevant_df.at[index,"Neutral Sentiment Score"] = s
    if i==2:
      relevant_df.at[index,"Negative Sentiment Score"] = s

In [35]:
print(relevant_df.groupby('publish_date')['Positive Sentiment Score'].mean())
print(relevant_df.groupby('publish_date')['Negative Sentiment Score'].mean())
print(relevant_df.groupby('publish_date')['Neutral Sentiment Score'].mean())

publish_date
20130101    0.070089
20130102    0.075063
20130103    0.069949
20130104    0.079223
20130105    0.077179
              ...   
20140511    0.079671
20140512    0.081984
20140513    0.077228
20140514    0.080468
20140515    0.072687
Name: Positive Sentiment Score, Length: 500, dtype: float64
publish_date
20130101    0.707103
20130102    0.708685
20130103    0.700789
20130104    0.706772
20130105    0.708572
              ...   
20140511    0.706995
20140512    0.698620
20140513    0.721527
20140514    0.709497
20140515    0.717883
Name: Negative Sentiment Score, Length: 500, dtype: float64
publish_date
20130101    0.222808
20130102    0.216252
20130103    0.229261
20130104    0.214005
20130105    0.214249
              ...   
20140511    0.213334
20140512    0.219396
20140513    0.201245
20140514    0.210035
20140515    0.209431
Name: Neutral Sentiment Score, Length: 500, dtype: float64


In [36]:
relevant_df["Entities"] = ""
relevant_df["Relations"] = ""
from spacy.matcher import Matcher 
from spacy.tokens import Span 

def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1",[pattern]) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

/var/folders/4p/rylg4l1j585g32zrfkvlzbgh0000gn/T/ipykernel_12510/934228981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_df["Entities"] = ""
/var/folders/4p/rylg4l1j585g32zrfkvlzbgh0000gn/T/ipykernel_12510/934228981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_df["Relations"] = ""


In [ ]:
nlp = en_core_web_sm.load()
total_nodes = []
entities = ["ORG", "PERSON", "NORP", "EVENT", "LAW", "MONEY"]
edge_info = []
node_dict = dict()


for index, row in relevant_df.iterrows():
    print(index)
    nodes = []

    article = nlp(row["headline_text"])

    for x in article.ents:
        if x.label_ in entities:
            nodes.append(x.text)
        if not node_dict.get(x.text):
            node_dict[x.text] = {"Positive Sentiment Score": row["Positive Sentiment Score"], 
                                 "Neutral Sentiment Score": row["Neutral Sentiment Score"],
                                 "Negative Sentiment Score": row["Negative Sentiment Score"],
                                 "count": 1}
        else:
            node_dict[x.text]["Positive Sentiment Score"] = node_dict[x.text]["Positive Sentiment Score"]+ row["Positive Sentiment Score"]
            node_dict[x.text]["Neutral Sentiment Score"] = node_dict[x.text]["Neutral Sentiment Score"]+ row["Neutral Sentiment Score"]
            node_dict[x.text]["Negative Sentiment Score"] = node_dict[x.text]["Negative Sentiment Score"]+  row["Negative Sentiment Score"]
            node_dict[x.text]["count"] = node_dict[x.text]["count"]+1
        
    
    relevant_df.at[index,"Entities"] = nodes
    relation = get_relation( row["headline_text"])

    relevant_df.at[index,"Relations"] = relation

    total_nodes = total_nodes+ nodes

    for i in range(len(nodes)):
        for j in range(i+1,len(nodes)):
            edge_info.append( ( nodes[i] , nodes[j]  ))

    

In [38]:
relevant_df.to_csv("check.csv")

In [39]:
print(len(set(total_nodes)))

6205


In [40]:
len(edge_info)

12001

In [41]:
len(node_dict)

9226

In [42]:
for n in total_nodes:
  node_dict[n]["Average Positive Sentiment Score"] = node_dict[n]["Positive Sentiment Score"]/node_dict[n]["count"]
  node_dict[n]["Average Neutral Sentiment Score"] = node_dict[n]["Neutral Sentiment Score"]/node_dict[n]["count"]
  node_dict[n]["Average Negative Sentiment Score"] = node_dict[n]["Negative Sentiment Score"]/node_dict[n]["count"]

In [43]:
graph_nodes = []
edges =[]

In [44]:
for n in set(total_nodes):
  graph_nodes.append((n, node_dict[n]['Average Positive Sentiment Score'],node_dict[n]['Average Neutral Sentiment Score'],node_dict[n]['Average Negative Sentiment Score']))

In [45]:
for e in edge_info:
  n1 = (e[0],node_dict[e[0]]['Average Positive Sentiment Score'],node_dict[e[0]]['Average Neutral Sentiment Score'],node_dict[e[0]]['Average Negative Sentiment Score'])
  n2 = (e[1],node_dict[e[1]]['Average Positive Sentiment Score'],node_dict[e[1]]['Average Neutral Sentiment Score'],node_dict[e[1]]['Average Negative Sentiment Score'])
  edges.append((n1,n2))

In [46]:
import networkx as nx
G = nx.Graph()

In [47]:
for n in graph_nodes:
  G.add_node(n)

In [48]:
G.add_edges_from(edges)

In [49]:
import pickle

# save graph object to file
pickle.dump(G, open('ToIGraph.pickle', 'wb'))



In [ ]:
import matplotlib.pyplot as plt



In [ ]:
nx.draw(G)

In [53]:
for n in graph_nodes:
  print(len(G.edges(n)))

1
0
2
0
1
0
0
0
1
1
6
1
1
3
1
7
1
0
1
4
0
0
3
1
0
0
1
0
1
0
1
3
0
1
33
8
0
0
1
0
0
1
4
1
1
0
0
0
1
0
1
4
0
0
6
5
0
2
0
0
0
1
0
4
0
3
0
1
0
2
1
2
1
2
0
1
1
1
2
0
1
1
5
9
2
1
1
2
2
0
1
3
0
6
1
9
0
5
1
1
1
0
0
2
0
0
0
3
1
23
1
1
0
2
0
1
0
2
10
0
0
1
1
3
7
1
5
1
1
1
1
4
4
2
1
1
2
1
1
1
1
0
0
1
1
0
2
0
0
15
3
4
1
1
2
1
1
1
1
5
3
1
0
0
1
0
1
1
1
0
0
2
1
1
0
0
2
1
5
0
0
1
0
0
2
0
0
0
2
1
0
52
0
0
0
0
1
1
1
1
1
0
1
1
2
0
0
1
0
1
4
0
0
0
1
0
1
0
4
0
3
2
1
1
0
0
0
3
2
1
0
0
0
0
5
0
0
1
0
1
1
1
1
1
1
1
1
0
1
1
2
0
4
0
2
0
1
1
2
1
3
1
1
2
7
1
0
0
0
2
0
1
0
12
0
0
0
0
1
2
1
1
5
32
0
1
3
0
1
0
2
0
1
5
2
1
2
1
0
0
2
4
1
0
2
1
1
0
0
0
2
5
0
0
1
1
1
1
0
0
0
0
1
1
1
0
1
4
2
1
1
1
0
0
1
8
0
0
2
0
0
5
0
4
0
0
6
0
0
0
1
1
0
0
149
0
1
1
1
3
0
0
1
1
3
0
4
2
0
0
0
0
2
4
13
2
0
0
1
1
0
0
0
4
0
0
0
0
0
1
1
6
1
10
6
3
2
2
1
0
2
1
1
8
1
1
1
3
1
2
6
1
1
2
0
0
0
1
1
0
2
0
0
2
4
1
4
1
9
2
1
1
0
2
1
2
2
1
3
3
1
4
0
0
1
4
1
2
3
0
0
1
1
1
1
1
2
3
0
4
0
0
2
1
1
0
2
0
2
3
23
0
3
0
0
0
0
3
1
1
11
0
5
0
0
0
5
9
0
1
3
0
2
0

In [50]:
# load graph object from file
graph = pickle.load(open('ToIGraph.pickle', 'rb'))

In [51]:
graph.edges

EdgeView([(('Republican Party of India', 0.029573490843176842, 0.25430524349212646, 0.7161213159561157), ('BJP-Shiv Sena', 0.03670132998377085, 0.18380635976791382, 0.7794923484325409)), (('FIA', 0.035031720995903015, 0.3569532036781311, 0.6080151200294495), ('Rehman Malik', 0.037832739452521004, 0.20983898639678955, 0.7523282965024313)), (('FIA', 0.035031720995903015, 0.3569532036781311, 0.6080151200294495), ('Indian', 0.0674025645282388, 0.23097080448306762, 0.7016266332687559)), (('Kalka-Shimla', 0.08464504188547532, 0.1489945873618126, 0.766360362370809), ('Himachal Pradesh', 0.062470033710055495, 0.21621631422400975, 0.7213136576303915)), (('LR', 0.044336527585983276, 0.3902008831501007, 0.565462589263916), ('Headley', 0.06445439470311005, 0.24207368679344654, 0.6934719234704971)), (('Mother Tamil', 0.03010169230401516, 0.3310862183570862, 0.6388120651245117), ('Jayalalithaa', 0.06876648996511231, 0.2226354241533124, 0.7085980881815371)), (('Operation Blue Star', 0.074924126267433